In [1]:
import pandas as pd
import numpy as np
from scipy import stats


In [5]:
df = pd.read_csv(
   "C:/Users/Abhi/Desktop/Python_Data_Analytics_Projects/content_engagement_churn_analytics/data/processed/churn_monetization_analysis.csv"
)

df.head()

,user_id,subscription_plan,churn_flag,avg_watch_minutes,avg_sessions,engagement_volatility,avg_watch_7d,avg_watch_30d,revenue_30d,engagement_trend,engagement_score,churn_risk_score,churn_risk_segment
0,U00001,Basic,1,122.327536,4.547826,71.764339,122.122885,122.184636,14.140580,0.143272,28.766061,24.727127,High Risk
1,U00002,Basic,1,121.375000,4.601562,63.786368,121.955407,121.521901,14.363281,-0.337561,30.794558,21.580623,Low Risk
2,U00003,Basic,1,124.544828,4.583908,67.809904,123.947411,122.452469,14.261494,0.251152,30.850132,22.739532,Medium Risk
3,U00004,Free,1,128.353247,4.927273,69.014366,128.166358,127.670915,5.371558,-0.169271,32.115171,22.570291,Medium Risk
4,U00005,Basic,1,121.992167,4.738903,63.305509,121.290066,120.139129,14.542428,0.096485,31.226885,21.191112,Low Risk


In [7]:
revenue_risk = (
    df.groupby("churn_risk_segment")
      .agg(
          users=("user_id", "count"),
          churn_rate=("churn_flag", "mean"),
          avg_revenue_30d=("revenue_30d", "mean"),
          total_revenue_30d=("revenue_30d", "sum")
      )
      .reset_index()
)

revenue_risk


,churn_risk_segment,users,churn_rate,avg_revenue_30d,total_revenue_30d
0,High Risk,400,1.000,11.297645,4519.058049
1,Low Risk,400,0.995,12.010652,4804.260606
2,Medium Risk,400,0.995,11.991397,4796.558843


In [9]:
churned_revenue = (
    df[df["churn_flag"] == 1]
    .groupby("subscription_plan")["revenue_30d"]
    .sum()
)

churned_revenue


subscription_plan
Basic      6082.242555
Free       2747.929819
Premium    5240.223141
Name: revenue_30d, dtype: float64

In [11]:
high_value_churn = df[
    (df["churn_flag"] == 1) &
    (df["revenue_30d"] > df["revenue_30d"].median())
]

high_value_churn[[
    "user_id",
    "subscription_plan",
    "revenue_30d",
    "churn_risk_segment",
    "engagement_score"
]].head()


,user_id,subscription_plan,revenue_30d,churn_risk_segment,engagement_score
0,U00001,Basic,14.140580,High Risk,28.766061
1,U00002,Basic,14.363281,Low Risk,30.794558
2,U00003,Basic,14.261494,Medium Risk,30.850132
4,U00005,Basic,14.542428,Low Risk,31.226885
6,U00007,Basic,14.644410,Medium Risk,30.938773


In [13]:
retention_matrix = (
    df.groupby(["churn_risk_segment", "subscription_plan"])
      .agg(
          users=("user_id", "count"),
          churn_rate=("churn_flag", "mean"),
          avg_revenue=("revenue_30d", "mean")
      )
      .reset_index()
)

retention_matrix


,churn_risk_segment,subscription_plan,users,churn_rate,avg_revenue
0,High Risk,Basic,140,1.000000,13.994850
1,High Risk,Free,189,1.000000,5.069366
2,High Risk,Premium,71,1.000000,22.558716
3,Low Risk,Basic,132,0.992424,14.192280
4,Low Risk,Free,181,0.994475,5.173433
5,Low Risk,Premium,87,1.000000,22.925153
6,Medium Risk,Basic,159,1.000000,14.243149
7,Medium Risk,Free,168,0.994048,5.143974
8,Medium Risk,Premium,73,0.986301,22.845350


In [15]:
df.to_csv(
    "C:/Users/Abhi/Desktop/Python_Data_Analytics_Projects/content_engagement_churn_analytics/data/processed/content_retention_impact_analysis.csv",
    index=False
)